In [54]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 
import imblearn


In [55]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

In [56]:
data = insp.merge(network, how='inner', on='PipeId')
data = data.drop(columns=["MaintenanceId","InspectionYear","InspectionDate","MonthsLastRev","Severity"], axis = 1)
data

,PipeId,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
1,189311802,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
2,189311802,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
3,189311802,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
4,189311802,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,0,Valencia,Betera,1993,PE,Gas natural,63.0,1.237,4.000,0,0,False
6345340,56907425,0,Valencia,Betera,2000,PE,Gas natural,160.0,169.732,4.000,1,0,False
6345341,7843615,0,Barcelona,Sabadell,1995,FD,Gas natural,150.0,0.361,0.025,0,0,True
6345342,56922456,0,Valencia,Betera,1993,PE,Gas natural,63.0,23.306,4.000,0,0,False


## Merging the datasets

In [57]:
finalsub = samplesub.merge(network, how='left', on='PipeId')
finalsub = finalsub.drop(columns="Incidence")
finalsub

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,446859944,Valencia,Alginet,2012,PE,Gas natural,110.0,44.286,0.150,0,0,False
1,428124500,Valencia,Alginet,2012,PE,Gas natural,160.0,110.715,0.150,2,0,False
2,438428871,Valencia,Alginet,2012,PE,Gas natural,110.0,13.575,0.150,0,0,False
3,429034569,Valencia,Alginet,2012,PE,Gas natural,110.0,327.689,0.150,3,0,False
4,411184477,Valencia,Alginet,2007,PE,Gas natural,200.0,932.762,0.150,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,Valencia,Paterna,2004,PE,Gas natural,110.0,7.656,0.025,0,0,False
909729,235426673,Valencia,Paterna,2004,PE,Gas natural,110.0,4.383,0.025,2,0,False
909730,235426707,Valencia,Paterna,2004,PE,Gas natural,110.0,2.292,0.025,1,0,False
909731,190956601,Valencia,Paterna,2004,PE,Gas natural,90.0,87.809,4.000,0,0,False


In [58]:
# df = data
# df = df.sample(frac=1)
# incidence_df = df.loc[df['Incidence'] == 1]
# non_incidence_df = df.loc[df['Incidence'] == 0][:20000]
# normal_distributed_df = pd.concat([incidence_df, non_incidence_df])
# new_df = normal_distributed_df.sample(frac=1, random_state=42)
# new_df.sample(3)

In [59]:
import pandas as pd
from sklearn.utils import resample

# Separate majority and minority classes
data_majority = data[data.Incidence==0]
data_minority = data[data.Incidence==1]

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(data_majority),    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
data_upsampled.Incidence.value_counts()

0    6333016
1    6333016
Name: Incidence, dtype: int64

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_upsampled.loc[:, data_upsampled.columns != "Incidence"], data_upsampled["Incidence"], test_size=0.1, random_state=42)

In [61]:
cat_features = ["Province","Town","Material","GasType","BoolBridle"]


In [62]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    #loss_function='CrossEntropy'
)


clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_test, y_test), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.1}


In [63]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=10,
#     verbose=5,
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
)

Learning rate set to 0.5
0:	learn: 0.5330903	test: 0.5332659	best: 0.5332659 (0)	total: 1.89s	remaining: 17s
1:	learn: 0.4705744	test: 0.4708504	best: 0.4708504 (1)	total: 3.45s	remaining: 13.8s
2:	learn: 0.4253966	test: 0.4257443	best: 0.4257443 (2)	total: 5.21s	remaining: 12.2s
3:	learn: 0.4032477	test: 0.4035805	best: 0.4035805 (3)	total: 6.63s	remaining: 9.94s
4:	learn: 0.3907613	test: 0.3910472	best: 0.3910472 (4)	total: 7.89s	remaining: 7.89s
5:	learn: 0.3844703	test: 0.3847550	best: 0.3847550 (5)	total: 9.18s	remaining: 6.12s
6:	learn: 0.3799457	test: 0.3801891	best: 0.3801891 (6)	total: 10.4s	remaining: 4.47s
7:	learn: 0.3772835	test: 0.3774764	best: 0.3774764 (7)	total: 11.6s	remaining: 2.91s
8:	learn: 0.3752977	test: 0.3755109	best: 0.3755109 (8)	total: 12.8s	remaining: 1.42s
9:	learn: 0.3707200	test: 0.3709302	best: 0.3709302 (9)	total: 13.9s	remaining: 0us

bestTest = 0.3709302109
bestIteration = 9



In [64]:
# Evaluate the model on the test set
results_1 = clf.predict(finalsub)
results1df = pd.DataFrame(results_1)

results1df = pd.DataFrame(results_1)

samplesub_sub = samplesub.drop(columns = "Incidence")

finalfinal = samplesub_sub.merge(results1df, left_index = True,right_index=True)
finalfinal.rename(columns={0: "Incidence"}, inplace=True)
finalfinal

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0
3,429034569,0
4,411184477,0
...,...,...
909728,235426649,0
909729,235426673,0
909730,235426707,0
909731,190956601,0


In [65]:
finalfinal.to_csv("CatBoostOversampling.csv", index=False)